In [1]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
from scipy.sparse import coo_matrix
import networkx as nx
import matplotlib.pyplot as plt
import folium
import warnings
import sys

warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

Geopandas has version 0.13.2
Movingpandas has version 0.17.1


In [2]:
# add paths for modules
sys.path.append('../src/models')
sys.path.append('../src/visualization')
sys.path.append('../src/features')
# import modules
import visualize
import geometry_utils
from maritime_traffic_network import MaritimeTrafficNetwork

['/Users/janhendrikwebert/maritime_route_prediction/notebooks', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python311.zip', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11/lib-dynload', '', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11/site-packages', '../src/models', '../src/visualization', '../src/features', '../visualization', '../features']


In [3]:
# read raw data from file
filename = '../data/processed/202204_points_stavanger_cleaned_meta_200k.parquet'
gdf = gpd.read_parquet(filename)
gdf.to_crs(32632, inplace=True)

In [4]:
gdf.crs

<Projected CRS: EPSG:32632>
Name: WGS 84 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 6°E and 12°E, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Austria. Cameroon. Denmark. Equatorial Guinea. France. Gabon. Germany. Italy. Libya. Liechtenstein. Monaco. Netherlands. Niger. Nigeria. Norway. Sao Tome and Principe. Svalbard. Sweden. Switzerland. Tunisia. Vatican City State.
- bounds: (6.0, 0.0, 12.0, 84.0)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
# initialize maritime traffic network
network = MaritimeTrafficNetwork(gdf)
network.get_trajectories_info()

AIS messages: 192346
Trajectories: 249


In [6]:
# compute Douglas Peucker significant points
network.calc_significant_points_DP(200)

Calculating significant turning points with Douglas Peucker algorithm (tolerance = 200) ...
Number of significant points detected: 2595 (1.35% of AIS messages)
Time elapsed: 0.11 minutes
Adding course over ground before and after each turn ...
Done. Time elapsed: 0.01 minutes


In [16]:
mmsi = '219347000_0'
trajectory = network.trajectories.get_trajectory(mmsi)
DP_trajectory = network.significant_points_trajectory.get_trajectory(mmsi)
nst = trajectory.to_traj_gdf().geometry.simplify(tolerance = 250)

In [17]:
nst

0    LINESTRING (326371.313 6468317.560, 322073.362...
dtype: geometry

In [18]:
#map = network.map_waypoints()
map = visualize.map_accurate_and_simplified_trajectory(trajectory, DP_trajectory)
map = nst.explore(m=map)
folium.LayerControl().add_to(map)
map

In [10]:
trajectory.get_crs()

<Projected CRS: EPSG:32632>
Name: WGS 84 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 6°E and 12°E, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Austria. Cameroon. Denmark. Equatorial Guinea. France. Gabon. Germany. Italy. Libya. Liechtenstein. Monaco. Netherlands. Niger. Nigeria. Norway. Sao Tome and Principe. Svalbard. Sweden. Switzerland. Tunisia. Vatican City State.
- bounds: (6.0, 0.0, 12.0, 84.0)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
network.significant_points

,mmsi,imo_nr,length,lon,lat,sog,cog,true_heading,nav_status,message_nr,bredde,dypgaaende,skipstype,skipsgruppe,fartoynavn,geometry,speed,cog_before,cog_after
date_time_utc,,,,,,,,,,,,,,,,,,,
2022-04-01 06:30:21,209989000_0,9235505,90,4.62360,59.5881,10.0,167.2,174,0,1,13.600000,6.36,General Cargo Ship,Last,LISA,POINT (252983.773 6613682.119),4.473722,176.570516,176.570516
2022-04-01 08:47:03,209989000_0,9235505,90,4.71718,59.2081,10.7,167.8,158,0,3,13.600000,6.36,General Cargo Ship,Last,LISA,POINT (255536.880 6571078.772),6.560909,176.570516,157.032343
2022-04-01 09:03:30,209989000_0,9235505,90,4.76105,59.1632,11.0,154.1,148,0,3,13.600000,6.36,General Cargo Ship,Last,LISA,POINT (257720.935 6565925.374),5.481145,157.032343,152.465795
2022-04-01 10:52:00,209989000_0,9235505,90,5.07398,58.8951,10.0,152.7,150,0,1,13.600000,6.36,General Cargo Ship,Last,LISA,POINT (273837.976 6535009.903),5.728811,152.465795,155.326025
2022-04-01 11:28:20,209989000_0,9235505,90,5.16866,58.8028,10.1,148.4,145,0,1,13.600000,6.36,General Cargo Ship,Last,LISA,POINT (278699.845 6524426.782),5.637932,155.326025,150.486590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-01 13:32:00,259216000_3,9344760,129,5.58913,59.1385,5.6,16.2,17,0,1,18.700001,4.60,Passenger/Ro-Ro Ship (Vehicles),Passasjer,MASTRAFJORD,POINT (304879.789 6560461.999),0.045701,160.205697,353.128081
2022-04-01 13:34:18,259216000_3,9344760,129,5.58681,59.1454,14.4,311.9,304,0,1,18.700001,4.60,Passenger/Ro-Ro Ship (Vehicles),Passasjer,MASTRAFJORD,POINT (304786.452 6561236.477),7.330095,353.128081,299.951784
2022-04-01 13:39:53,259216000_3,9344760,129,5.54517,59.1563,15.6,296.4,295,0,1,18.700001,4.60,Passenger/Ro-Ro Ship (Vehicles),Passasjer,MASTRAFJORD,POINT (302469.115 6562571.793),10.071801,299.951784,296.325745
